<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_Comparativa_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
EVALUACIÓN COMPARATIVA. Mask2former vs OneFormer para Segmentación de personas.

Se proporcionar análisis detallado de las ventajas de la segmentación panóptica
con OneFormer frente segmentación de instancias tradicional.

Conceptos clave:
- Segmentación panóptica: Segmentación semántica e instancias
- Comparación arquitectónica: Transformer-based models estado del arte
- Evaluación académica: Métricas comprehensivas.

Autor: Jesús L.
Trabajo: Evaluación comparativa de técnicas de segmentación.
Fecha: Agosto 2025.

Referencias Técnicas:
- Cheng et al. "Masked-attention Mask Transformer for Universal Image Segmentation" (Mask2Former)
- Jain et al. "OneFormer: One Transformer to Rule Universal Image Segmentation" (OneFormer)
- Kirillov et al. "Panoptic Segmentation" (Conceptos fundamentales)
"""

'\nEVALUACIÓN COMPARATIVA. Mask2former vs OneFormer para Segmentación de personas.\n\nSe proporcionar análisis detallado de las ventajas de la segmentación panóptica\ncon OneFormer frente segmentación de instancias tradicional.\n\nConceptos clave:\n- Segmentación panóptica: Segmentación semántica e instancias\n- Comparación arquitectónica: Transformer-based models estado del arte\n- Evaluación académica: Métricas comprehensivas.\n\nAutor: Jesús L.\nTrabajo: Evaluación comparativa de técnicas de segmentación.\nFecha: Agosto 2025.\n\nReferencias Técnicas:\n- Cheng et al. "Masked-attention Mask Transformer for Universal Image Segmentation" (Mask2Former)\n- Jain et al. "OneFormer: One Transformer to Rule Universal Image Segmentation" (OneFormer)\n- Kirillov et al. "Panoptic Segmentation" (Conceptos fundamentales)\n'

In [6]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 14.6 MB/s eta 0:00:00


In [3]:
import os
import torch
import logging
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Union, Literal
from dataclasses import dataclass
from pathlib import Path
import json
import cv2
from datetime import datetime
import traceback
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Importaciones específicas de Hugging Face para ambos modelos
from transformers import (
    # Mask2Former components
    Mask2FormerImageProcessor,
    Mask2FormerForUniversalSegmentation,
    # OneFormer components
    OneFormerImageProcessor,
    OneFormerForUniversalSegmentation,
    # Generic auto-loading
    AutoImageProcessor,
    AutoModelForUniversalSegmentation
)

In [8]:
# Métricas y utilidades para evaluación
from sklearn.metrics import precision_recall_fscore_support, jaccard_score
import torchmetrics
from torchmetrics.segmentation import MeanIoU, DiceScore
from torchmetrics.classification import BinaryJaccardIndex

In [9]:
# Utilidades para procesamiento y visualización
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [11]:
@dataclass
class ComparativeEvaluationConfig:
  """
    Evaluación comparativa de modelos de segmentación.

    + Atributos de configuración de modelos:
    ++models_to_evaluate(List[str]): Lista de modelos que vamos a comparar.
      Opciones posibles:
       - "facebook/mask2former-swin-base-coco-panoptic": Mask2Former baseline
       - "shi-labs/oneformer_coco_swin_large": OneFormer optimizado para COCO
       - "shi-labs/oneformer_ade20k_swin_large": OneFormer para escenas complejas
    ++confidence_threshold(float): Umbral mínimo de confianza para aceptar detecciones.
      Rango [0.0,1.0] para personas, valores óptimos entre 0.6 y 0.8 balancean
      precisión y recall. Valor único aplicado para ambos modelos de comparación justa.
    ++segmentation_task(str): Tipo de segmentación a evaluar específicamente.
      - "panoptic": Segmentación panóptica. Recomendado para OneFormer.
      - "instance": Segmentación de instancias. Comparación directa.
      - "semantic": Segmentación semántica. Recomendado para Mask2Former. Análisis
      de precisión.

    + Atributos de configuración de datos:
    ++images_directory(str): Directorio raíz de con imágenes de evaluación en formatos
    estándar (JPG,PNG,BMP).
    ++ground_truth_masks_dir(str): Directorio con máscaras de verdad fundamental
    en formato PNG binario (0=fondo, 255=persona). Esto es opcional si solo se
    requiere inferencia sin evaluación cuantitativa.
    ++panoptic_annotations_file(str): Fichero COCO-format JSON con anotaciones
    panópticas para evaluación completa. Incluye información semántica e instancias.
    ++output_directory(str): Directorio de salida para todos los resultados.
    Se crea automáticamente con subdirectorios organizados por modelo.

    + Atributos de configuración de procesamiento:
    ++batch_size(int): Número de imágenes procesadas simultáneamente por el modelo.
    Este valor lo debemos ajustar dependiendo del entorno.
    ++max_images(int): Límite de imágenes para evaluación rápida.
    None para evaluar todas las imágenes.
    ++resize_images(bool): Si debemos redimensionar imágenes para eficiencia de
    cómputo. Aquí será "True" si tenemos dataset con imágenes de alta resolución.
    ++target_size(Tuple[int,int]): Dimensiones objetivo (altura, ancho) para
    redimensionar. Debe ser múltiplo de 32 para que sea eficiente y óptimo
    de la arquitectura transformer.
    ++enable_mixed_precision(bool): Si denbemos utilizar precisión mixta para
    acelerar inferencia en GPUs compatibles. Reduce uso de memoria manteniendo
    precisión.

    + Atributos de configuración de análisis:
    ++compare_architectures(bool): Para realizar análisis comparativo detallado
    entre arquitecturas. Incluyo análisis de complejidad y eficiencia.
    ++analyze_panoptic_advantages(bool): Si evaluamos específicamente ventajas
    de segmentación panóptica sobre instancias.
    ++save_detailed_predictions(bool): Para crear visualizaciones lado a lado
    comparando resultados de los modelos.
    ++visualization_samples(int): Número de muestras visuales a generar.
    Balance entre utilidad analítica y uso de almacenamiento.
    ++detailed_logging(bool): Para activar logging exhaustivo para la documentación
    y reproducibilidad de los experimentos.

  """

  #Configuración de modelos para comparación.
  models_to_evaluate: List[str] = None
  confidence_threshold: float = 0.7
  segmentation_task: Literal["panoptic", "instance", "semantic"] = "panoptic"
  device: str = "auto"

  #Configuración de datos de entrada.
  images_directory: str = ""
  ground_truth_masks_dir: str = ""
  panoptic_annotations_file: str = ""
  output_directory: str = "comparative_segmentation_results"

  #Configuración de procesamiento optimizado.
  batch_size: int = 1 ## Revisar. Es muy conservador.
  max_images: Optional[int] = None
  resize_images: bool = True
  target_size: Tuple[int,int] = (512,512) ## Eficiente para transformers.
  enable_mixed_precision: bool = True

  #Configuración de análisis comparativo.
  compare_architectures: bool = True
  analyze_panoptic_advantages: bool = True
  save_detailed_predictions: bool = True
  visualization_samples: int = 15
  detailed_logging: bool = True

  def __post_init__(self):
    """
      Configuración automática y validación de parámetros.
    """
    #Configuración automática si no se define.
    if self.models_to_evaluate is None:
      self.models_to_evaluate = ["facebook/mask2former-swin-base-coco-panoptic",
                                  "shi-labs/oneformer_coco_swin_large",
                                  "shi-labs/oneformer_ade20k_swin_large"]

    #Detección automática de dispositivo.
    if self.device == "auto":
      if torch.cuda.is_available():
        self.device = "cuda"
      else: "cpu"

    #Validación de directorio de imágenes.
    if not self.images_directory:
        raise ValueError("Debe proporcionar un directorio de imágenes.")

    #Crear directorio de salida.
    os.makedirs(self.output_directory, exist_ok=True)

    #Ajustar configuración según dispositivo disponible.
    if self.device == "cpu":
        self.enable_mixed_precision == False ##Solo en GPU.
        self.batch_size = 1 ##Muy conservador en CPU.